# ac(adding calculator) 어휘 분석기 

All rights reserved, 2021-2023. By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 강의 노트 **ch2-1, ch2-2** 내용을 충분히 들여다 보기 바랍니다.
- 구체적인 코드를 이해할 필요는 없습니다. 나무를 보려는 게 아니라 숲을 보려고 하기 때문이죠.
    - Try to see the forest, not for the trees!
- 컴파일러 전반부(front-end)의 구현 과정에 대한 감을 잡기위한 것입니다.
    - nltk에서 제공하는 라이브러리를 사용하면 훨씬 간단하게 구현할 수 있습니다.
    - 그런데, 그건 구현이 아니라 라이브러리 사용 방법을 익히는 것일 뿐입니다.
    - 이 과목을 수강하지 않더라도 누구나 그 정도는 할 수 있습니다.
- 여러분도 nltk에서 제공하는 라이브러리 정도는 직접 만들어, github에서 제공하고 싶지 않나요?

In [1]:
class Token:
    def __init__(self, typ, val):
        self.type = typ
        self.value = val  

토큰 *type*은 dictionary 객체로 정의할 수 있습니다.

In [2]:
TOKENS = {
    'ID':0, 'FLTDCL':1, 'INTDCL':2, 
    'PRINT':3, 'ASSIGN':4, 'PLUS':5, 
    'MINUS':6, 'INUM':7, 'FNUM':8, 
    'EOF':9, 'ERROR':10
}

- 이름(identifier)을 저장하기 위한 symbol table은 아래와 같이 만들 수 있습니다.
    - **ac** 언어에서는 알파벳 26자 중 23개의 변수를 정의할 수 있습니다.
    - *f, i, p*는 **keyword**이기 때문에 변수로 사용할 수 없습니다.

In [3]:
import string

alphabet = string.ascii_lowercase
print(type(alphabet))

symbol_table = {}  
for i in range(len(alphabet)):
    symbol_table[alphabet[i]] = 0

symbol_table['f'] = symbol_table['i'] = symbol_table['p'] = None
print(symbol_table['a'])
print(symbol_table['f'])

<class 'str'>
0
None


ac 언어 구문에 맞는 문장을 *istream*에 할당하겠습니다.

In [4]:
istream = "f b   i a   a = 5   b = a + 3.2   p b "
print(type(istream), len(istream))

<class 'str'> 38


**peek()** 함수는 인덱스 *i*가 가리키는 문자 한 개를 읽어옵니다.

In [5]:
def peek(i):
    return istream[i] 

- **advance()** 함수는 인덱스 *i*를 한 칸 이동시켜, 다음 문자 한 개를 읽어옵니다.
    - 배열의 범위를 벗어나지 않도록 if-문을 사용했습니다. 
        - 배열의 크기를 *length* 라고 하면, 
        - 배열은 인덱스 0부터 시작하기 때문에 마지막 인덱스는 (*length-1*) 입니다.
    - 파이썬의 함수는 한 번에 2개 값을 반환할 수 있습니다.

In [6]:
def advance(i, lim):
    i += 1
    if (i < lim):
        s = istream[i]  
    else: 
        s = None
    return i, s

- **ac** 언어에서 가장 찾기 힘든 token은 숫자입니다.
    - 정수(INUM)인지 실수(FNUM)인지 구분하는 기준은 소숫점(\.)입니다.
- **ScanDigit()** 함수는 소숫점이 있으면 실수로 인식합니다. 
    - 패턴을 정의하는 정규표현을 사용하면 훨씬 다양한 형태의 실수를 인식할 수 있습니다.
    - 아래 코드에서 문자(char)끼리 더해(concatenate) 문자열(str)을 만드는 것에 주목하기 바랍니다.

In [7]:
def ScanDigit(idx):
    val = ""  #빈 문자열(str), 초기화.     
    s = peek(idx)
    limit = len(istream)
           
    while s.isdigit():
        val += s
        idx, s = advance(idx, limit)
        if s == None:
            return idx, Token('INUM', val)
    
    if (idx < limit and s != '.'):
        type = 'INUM'
    else:    
        type = 'FNUM'
        val += s     
        idx, s = advance(idx, limit)     
        if s == None:
            return idx, Token(type, val)            
        
        while s.isdigit():
            val += s  
            idx, s = advance(idx, limit)                     
            if s == None:
                break   
                    
    return idx, Token(type, val)   

- 아래처럼 숫자 예를 만들어 제대로 동작하는지 확인해 보겠습니다.
    - 마지막 007은 정수로 인식했지만 조금 이상하죠...

In [8]:
test_digits = ["32.572", "32", "007"]

for i in range(len(test_digits)):
    istream = test_digits[i]
    index, tok = ScanDigit(0)
    print(index, tok.type, tok.value)

6 FNUM 32.572
2 INUM 32
3 INUM 007


## Quiz 1
- 숫자 예를 3개 이상 만들어서 테스트해 보세요.

In [9]:
test_digits = ["-32", "01", "0.230"]

for i in range(len(test_digits)):
    istream = test_digits[i]
    index, tok = ScanDigit(0)
    print(index, tok.type, tok.value)

0 INUM 
2 INUM 01
5 FNUM 0.230


- 음수를 입력했을 경우에는 제대로 동작하지 않음
- 007 예시와 동일하게 01처럼 앞에 0이 붙은 정수의 경우 정수로 인식하지만 살짝 이상하게 인식함

영어 알파벳이 키워드 *f, i, p*인지 변수 이름(*a, b, c, ..., z*)인지 구분하는 함수를 만들겠습니다.

In [10]:
def representativeChar(c):
    if c.isalpha():
        if c not in ['f', 'i', 'p']:
            return True
    return False

- 이제 어휘분석기(**Scanner**)를 만들어 보겠습니다.
    - 어휘분석기는 한 번 호출될 때마다 하나의 토큰을 찾습니다.
    - **ac**언어에서 사용하는 기호는 의미가 미리 정해져 있습니다.
        - 사실 숫자를 찾는 것을 제외하면 간단한 코드입니다.
        - 파이썬에서는 *switch-case* 문이 없기 때문에 *if-elif-else* 문으로 작성했습니다.

In [11]:
def Scanner(idx):
    limit = len(istream)  
    val = ""    
    ans = Token('EOF', None)
    
    if idx >= limit-1:
        return idx, ans
    
    s = peek(idx)    
    while s == ' ':
        idx, s = advance(idx, limit)     
    
    if s != None:
        if s.isdigit():
            idx, ans = ScanDigit(idx)
        else:
            if representativeChar(s):
                ans = Token('ID', s)            
            elif s == 'f':
                ans = Token('FLTDCL', None)
            elif s == 'i':
                ans = Token('INTDCL', None)
            elif s == 'p':
                ans = Token('PRINT', None)
            elif s == '=':
                ans = Token('ASSIGN', None)
            elif s == '+':
                ans = Token('PLUS', None)
            elif s == '-':
                ans = Token('MINUS', None)   
            else:
                ans = Token('ERROR', s)
    
    return idx, ans

- 이제 어휘분석기가 token을 제대로 가져오는지 확인해 보겠습니다.
    - 문장 끝에 에러에 해당되는 문자를 넣어 error도 잘 찾는지도 테스트해 보겠습니다.

In [12]:
istream = "f b   i a   a = 5   b = a + 3.2   p b ?"
limit = len(istream)
index = 0

while index < limit:
    index, tok = Scanner(index)
    if tok.type == 'ERROR':
        print('ERROR >>> unexpected char', tok.value)
        break
        
    print(index, tok.type, tok.value)
    index, s = advance(index, limit)

0 FLTDCL None
2 ID b
6 INTDCL None
8 ID a
12 ID a
14 ASSIGN None
17 INUM 5
20 ID b
22 ASSIGN None
24 ID a
26 PLUS None
31 FNUM 3.2
34 PRINT None
36 ID b
ERROR >>> unexpected char ?
